In [5]:
##Libraries##

import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import insert
import pyodbc

def ContraValor (mont,precio):
    result = 0
    
    sidechoice = input("Which ccy do you have: 'crc or usd?")
    
    if sidechoice == 'usd':
        result = mont * precio
    elif sidechoice == 'crc':
        result = mont / precio
    else: 
        print("you have to chose a valid ccy") 
    return result    


In [15]:
##Connection to cmv_uat database##

server = 'tcp:mycmvserver.database.windows.net' 
database = 'cmvdb_uat' 
username = 'ecervantes@cmvlatam.com' 
password = 'Columbia87/' 


connection = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};Server=tcp:mycmvserver.database.windows.net,1433;Database=cmvdb_uat;Uid='+username+';Pwd='+password+';Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;Authentication=ActiveDirectoryInteractive')
cursor = connection.cursor()

In [11]:
pxcv = 615
montocv = 1000
clientecv = 116347
print(ContraValor(pxcv,montocv))


0.615


In [ ]:
### Trade Form ### ### play to fill details ###

sidechoiceraw,  = input("Which side bid=1 or ask=2?")
sidechoice = int(sidechoiceraw)
pxraw =  input ("precio a cliente?")
px = float(pxraw) 
monedatengoraw = input ("USD=1 CRC=2 EUR=5")
monedatengo = int(monedatengoraw)   
montoraw = input ("digite el monto de la moneda que tiene")
monto = float(montoraw)
clienteraw = input ("numero de cliente")
cliente = int(clienteraw)

print1 ="Bid or Ask:  {:,} "
print(print1.format(sidechoice))
print2 ="Precio a Cliente:  {:,} "
print(print2.format(px))
print3 ="moneda input:  {:,} "
print(print3.format(monedatengo))
print4 ="monto moneda input :  {:,} "
print(print4.format(monto))    
print5 ="N. de cliente:  {:,} "
print(print5.format(cliente))


In [52]:
##Market Data##
monex_pxlast = '619.15'                 ##last traded price in Monex--
monex_pp = '619.11'                     ##last update weighted average Monex--
px_bid_cover = '618.750'                     ##internal price split--
px_ask_cover = '619.750' 

In [ ]:
## Play to print ticket to DB ##
##Fix setup computations##
sideid = f"{sidechoice}"                            ## 1 = bid  /  2 = ask
clienteid = f"{cliente}"                    ##de 6 digitos nada mas-- 
px_sp = f"{px}"                        ##precio sport --
accionid = '1'                                ## Add New = 1 , Delete = 2 , reinstate = 3 -- 3 under dev ##
#tradedate = 'GETDATE()'                      ##date
#valuedate = 'GETDATE()'                      ##date 
estadoid = '1'                                ##aplicada, pendiente, anulada--
dealerid = '2'                                ##trader that inputs the transaction-- 
productotipoid = '1'                          ##spot o forward-- 
#solicitud = '400113'
deletetradeid = ''                            ##Trade ID del trade a borrar--

if sideid == '1' :
    comccyid = '1'                            ##Moneda a comprar USD=1 CRC=2 EUR=5--                              
    venccyid = '2'
    px_cover = px_bid_cover                            ##moneda a vender USD=1 CRC=2 EUR=5--
else : 
    comccyid = '2'                            ##Moneda a comprar USD=1 CRC=2 EUR=5--                              
    venccyid = '1'                            ##moneda a vender USD=1 CRC=2 EUR=5--
    px_cover = px_ask_cover

###El blotter no esta cambiando la moneda a la hora de hacer el print
if (sideid == '1' ) and (monedatengo == 1 ):  ##Es un Bid y Moneda input es USD=1##    
    montocom = monto
    montoven = px * monto
elif (sideid == '1') and (monedatengo == 2): ##Es un Bid y Moneda input es CRC=2##
    montocom = monto / px
    montoven = monto
elif (sideid == '2') and (monedatengo == 1):  ##Es un Ask y Moneda input es USD=1##
    montocom = px * monto
    montoven = monto
elif (sideid == '2') and (monedatengo == 2):  ##Es un Ask y Moneda input es CRC=2##
    montocom = monto
    montoven = monto / px    
else :
    print("ccy conbination not supported")    

montocom = f"{montocom}" ##monto a comprar, expresado en moneda de compra--
montoven = f"{montoven}" 


### Working ticket entry from Python--must link to variables on top ####
if accionid == 2 :
    SQLCommand = ('INSERT INTO [dbo].[fxblotter.t] ( [createdat], [accionid], [sideid], [clienteid], [comccyid], [montocom], [px_sp], [venccyid], [montoven],  [tradedate], [valuedate], [px_cover], [monex_pxlast], [monex_pp], [estadoid], [dealerid], [productotipoid], [solicitud], [deletetradeid]) VALUES (GETDATE(),'+accionid+','+sideid+','+clienteid+','+comccyid+','+montocom+','+px_sp+','+venccyid+','+montoven+',GETDATE(),GETDATE(),'+px_cover+','+monex_pxlast+','+monex_pp+','+estadoid+','+dealerid+','+productotipoid+', 0,'+deletetradeid+');')
    cursor.execute(SQLCommand)
    connection.commit()
    print(f"deletion of Trade"+deletetradeid+"printed ok")

else :
    SQLCommand = ('INSERT INTO [dbo].[fxblotter.t] ( [createdat], [accionid], [sideid], [clienteid], [comccyid], [montocom], [px_sp], [venccyid], [montoven],  [tradedate], [valuedate], [px_cover], [monex_pxlast], [monex_pp], [estadoid], [dealerid], [productotipoid], [solicitud]) VALUES (GETDATE(),'+accionid+','+sideid+','+clienteid+','+comccyid+','+montocom+','+px_sp+','+venccyid+','+montoven+',GETDATE(),GETDATE(),'+px_cover+','+monex_pxlast+','+monex_pp+','+estadoid+','+dealerid+','+productotipoid+', 0);')
    cursor.execute(SQLCommand)
    connection.commit()
    print(f"Trade printed ok")